* Integral the wind stress at Sandheads along- and cross-strait for different months (Jan, Oct and May).

In [1]:
import numpy as np
import netCDF4 as nc
from salishsea_tools import (nc_tools,viz_tools,tidetools,geo_tools)
import datetime, os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import wind
import pandas as pd
%matplotlib inline
sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=2, rc={"lines.linewidth": 2.5})

In [2]:
transects = {'1':{'jst':55, 'ist':25,'jen':56,'ien':80},
            '2':{'jst':25, 'ist':25,'jen':26,'ien':75},
            '3':{'jst':25, 'ist':25,'jen':56,'ien':26},
            '4':{'jst':37, 'ist':72,'jen':45,'ien':73},
            '5':{'jst':120, 'ist':138,'jen':121,'ien':139},
            '6':{'jst':55,  'ist': 1,'jen':56,'ien':25},
            '7':{'jst':25,'ist':1, 'jen':26,'ien':25}}

In [3]:
grid6 = nc.Dataset('/ocean/jieliu/research/meopar/river-treatment/bathy_meter_SalishSea6.nc')
X = grid6.variables['nav_lon'][380:510,240:397]
Y = grid6.variables['nav_lat'][380:510,240:397]
bathy = grid6.variables['Bathymetry'][380:510,240:397]

In [4]:
## mask
mesh = nc.Dataset('/data/jieliu/MEOPAR/river-treatment/oct8_101e061e05/mesh_mask.nc')
tmask = mesh.variables['tmask'][0,:,380:510,240:397]
e3t = mesh.variables['e3t'][0,:,380:510,240:397]
e2t = mesh.variables['e2t'][0,380:510,240:397]
e1t = mesh.variables['e1t'][0,380:510,240:397]
np_mask = np.abs(1-tmask) 

In [5]:
def get_OPwinds(stationsOP,OPlon,OPlat,filesOP,j,i,duration,sax,eax):
    "This function was made to get model winds' speed and direction"
    wind_speed = {}; wind_dir = {}; time = {}; lat={}; lon={}; press={}; temp={}; therm={};
    solar={}; precip={}; qair={}; lon = {};lat = {};

    for  modOP in stationsOP:   
        
        [j,i]=wind.find_model_point(X[j,i],Y[j,i],OPlon,OPlat)
        lon[modOP] = OPlon[j,i]
        lat[modOP]=OPlat[j,i]
        print(j,i,lon[modOP],lat[modOP])

        [wind_speed[modOP],wind_dir[modOP],time[modOP],
         press[modOP],temp[modOP],solar[modOP],
         therm[modOP],qair[modOP],precip[modOP]] = wind.compile_OP(filesOP,j,i)    
      
    return wind_speed, wind_dir

In [9]:
def Integral_Velocity(hours,v_may):
    """
    To integral any velocity component in hours we define
    """
    v_integral = np.zeros(v_may.shape[1]/hours)
    j = 0
    for i in np.arange(0,v_may.shape[1],hours):
        v_integral[j] = np.sum(v_may[0,i:i+hours])
        j = j +1
    return v_integral

In [6]:
import glob
X = grid6.variables['nav_lon']
Y = grid6.variables['nav_lat']
OP = nc.Dataset('/results/forcing/atmospheric/GEM2.5/operational/ops_y2014m10d01.nc')
OPlon =OP.variables['nav_lon'][:]-360
OPlat = OP.variables['nav_lat'][:]
filesOP = glob.glob('/results/forcing/atmospheric/GEM2.5/operational/ops_y2015m05d*.nc')
filesOP.sort(key=os.path.basename)
start = '1-May-2015'; end = '31-May-2015';
start2 = '31-May-2015'; end2 = '31-May-2015';
endd = '31-May-2015';
stations = ['Sandheads']; stationsOP =['Sandheads_OP'];
sax=datetime.datetime.strptime(start,'%d-%b-%Y')
eax=datetime.datetime.strptime(endd,'%d-%b-%Y')

wind_speedMay,wind_dirMay =get_OPwinds(stationsOP,OPlon,OPlat,filesOP,426,293,\
                       27,sax,eax)

135 151 -123.302707487 49.0907848354


In [7]:
## May
wind_dir_correctMay = []
for dire in wind_dirMay['Sandheads_OP']:
    dire_corre = 450 - dire
    wind_dir_correctMay.append(dire_corre)
df_may = pd.DataFrame({"speed": wind_speedMay['Sandheads_OP'], "direction": wind_dir_correctMay})

In [13]:
CrossU_may = np.zeros([1,len(df_may)])
AlongV_may = np.zeros([1,len(df_may)])
for i in np.arange(len(df_may)):
    u = df_may.speed[i] * np.cos((450-df_may.direction[i])*np.pi/180) ## eastward wind 
    v = df_may.speed[i] * np.sin((450-df_may.direction[i])*np.pi/180) ## northward wind 
    #rotate
    theta = np.pi*29/180
    CrossU_may[0,i] = u*np.cos(theta) + v*np.sin(theta)
    AlongV_may[0,i] = -u*np.sin(theta) +v*np.cos(theta) 

In [14]:
CU_may_inte = Integral_Velocity(24,CrossU_may)
np.sum(CU_may_inte)

/ocean/jieliu/Anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


120.3658937535092

In [15]:
AV_may_inte = Integral_Velocity(24,AlongV_may)
np.sum(AV_may_inte)

/ocean/jieliu/Anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


37.6819473241124

In [16]:
filesOP = glob.glob('/results/forcing/atmospheric/GEM2.5/operational/ops_y2016m01d*.nc')
filesOP.sort(key=os.path.basename)
start = '1-Jan-2016'; end = '31-Jan-2016';
start2 = '31-Jan-2016'; end2 = '31-Jan-2016';
endd = '31-Jan-2016';
stations = ['Sandheads']; stationsOP =['Sandheads_OP'];
sax=datetime.datetime.strptime(start,'%d-%b-%Y')
eax=datetime.datetime.strptime(endd,'%d-%b-%Y')

wind_speedJan,wind_dirJan =get_OPwinds(stationsOP,OPlon,OPlat,filesOP,426,293,\
                       27,sax,eax)

135 151 -123.302707487 49.0907848354


In [17]:
## Jan
wind_dir_correctJan = []
for dire in wind_dirJan['Sandheads_OP']:
    dire_corre =  450 - dire 
    wind_dir_correctJan.append(dire_corre)
df_jan = pd.DataFrame({"speed": wind_speedJan['Sandheads_OP'], "direction": wind_dir_correctJan})

In [18]:
CrossU_jan = np.zeros([1,len(df_jan)])
AlongV_jan = np.zeros([1,len(df_jan)])
for i in np.arange(len(df_jan)):
    u = df_jan.speed[i] * np.cos((450-df_jan.direction[i])*np.pi/180)
    v = df_jan.speed[i] * np.sin((450-df_jan.direction[i])*np.pi/180)
    #rotate
    theta = np.pi*29/180
    CrossU_jan[0,i] = u*np.cos(theta) + v*np.sin(theta)
    AlongV_jan[0,i] = -u*np.sin(theta) +v*np.cos(theta) 

In [19]:
CU_jan_inte = Integral_Velocity(24,CrossU_jan)
np.sum(CU_jan_inte)

/ocean/jieliu/Anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


-753.92639700723043

In [20]:
AV_jan_inte = Integral_Velocity(24,AlongV_jan)
np.sum(AV_jan_inte)

/ocean/jieliu/Anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1387.3010893099661

In [21]:
## Oct
filesOP = glob.glob('/results/forcing/atmospheric/GEM2.5/operational/ops_y2014m10d*.nc')
filesOP.sort(key=os.path.basename)

OP = nc.Dataset('/results/forcing/atmospheric/GEM2.5/operational/ops_y2014m10d01.nc')
OPlon =OP.variables['nav_lon'][:]-360
OPlat = OP.variables['nav_lat'][:]

start = '1-Oct-2014'; end = '31-Oct-2014';
start2 = '31-Oct-2014'; end2 = '31-Oct-2014';
endd = '31-Oct-2014';
stations = ['Sandheads']; stationsOP =['Sandheads_OP'];
sax=datetime.datetime.strptime(start,'%d-%b-%Y')
eax=datetime.datetime.strptime(endd,'%d-%b-%Y')

wind_speed,wind_dir =get_OPwinds(stationsOP,OPlon,OPlat,filesOP,426,293,\
                            27,sax,eax)

135 151 -123.302707487 49.0907848354


In [22]:
## Oct
wind_dir_correct = []
for dire in wind_dir['Sandheads_OP']:
    dire_corre =  450 - dire
    wind_dir_correct.append(dire_corre)
## load into a pandas frame
df_oct = pd.DataFrame({"speed": wind_speed['Sandheads_OP'], "direction": wind_dir_correct})

In [23]:
CrossU_oct = np.zeros([1,len(df_oct)])
AlongV_oct = np.zeros([1,len(df_oct)])
for i in np.arange(len(df_oct)):
    u = df_oct.speed[i] * np.cos((450-df_oct.direction[i])*np.pi/180)
    v = df_oct.speed[i] * np.sin((450-df_oct.direction[i])*np.pi/180)
    #rotate
    theta = np.pi*29/180
    CrossU_oct[0,i] = u*np.cos(theta) + v*np.sin(theta)
    AlongV_oct[0,i] = -u*np.sin(theta) +v*np.cos(theta) 

In [24]:
CU_oct_inte = Integral_Velocity(24,CrossU_oct)
np.sum(CU_oct_inte)

/ocean/jieliu/Anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


-509.6422162608427

In [25]:
AV_oct_inte = Integral_Velocity(24,AlongV_oct)
np.sum(AV_oct_inte)

/ocean/jieliu/Anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1548.2755867583935